<a href="https://colab.research.google.com/github/Spidy104/Random-Assignment--DL/blob/main/Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [2]:
# Select device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Define transforms with data augmentation for training
transform_train = transforms.Compose([
    transforms.RandomRotation(10),           # Rotate images up to 10 degrees
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation
    transforms.ToTensor(),                   # Convert to tensor and normalize to [0,1]
    transforms.Normalize((0.1307,), (0.3081,)),  # Standardize with MNIST-like stats
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

# Load EMNIST-Letters dataset
train_dataset_full = torchvision.datasets.EMNIST(
    root='./data', split='letters', train=True, download=True, transform=None
)
test_dataset = torchvision.datasets.EMNIST(
    root='./data', split='letters', train=False, download=True, transform=transform_test
)

# Split training data into train and validation sets (90% train, 10% val)
train_size = int(0.9 * len(train_dataset_full))
val_size = len(train_dataset_full) - train_size
train_subset, val_subset = random_split(train_dataset_full, [train_size, val_size])


100%|██████████| 562M/562M [00:05<00:00, 109MB/s]


In [4]:
# Custom dataset class to apply transforms to subsets
class TransformedDataset(Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        img, label = self.subset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Apply transforms to train and val datasets
train_dataset = TransformedDataset(train_subset, transform_train)
val_dataset = TransformedDataset(val_subset, transform_test)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [5]:
# Function to adjust labels (1-26 to 0-25)
def adjust_labels(labels):
    return labels - 1

# Define the optimized CNN model
class OptimizedLetterCNN(nn.Module):
    def __init__(self):
        super(OptimizedLetterCNN, self).__init__()
        # Block 1: 2 conv layers with 32 filters
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 28x28 -> 14x14
        )
        # Block 2: 2 conv layers with 64 filters
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 14x14 -> 7x7
        )
        # Block 3: 1 conv layer with 128 filters
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4)),  # 7x7 -> 4x4
        )
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 26),  # 26 classes for letters A-Z
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.fc(x)
        return x


In [7]:
# Initialize model, criterion, optimizer, and scheduler
model = OptimizedLetterCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

In [8]:
# Training and evaluation functions
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), adjust_labels(labels).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return loss, accuracy

def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), adjust_labels(labels).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return loss, accuracy


In [9]:
# Training loop with early stopping
num_epochs = 50
best_val_acc = 0.0
patience = 10
trigger_times = 0
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Update learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping based on validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping!")
            break

# Load best model and evaluate on test set
model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

Epoch 1/50, Train Loss: 0.5240, Train Acc: 83.31%, Val Loss: 0.2483, Val Acc: 91.71%
Epoch 2/50, Train Loss: 0.2806, Train Acc: 90.72%, Val Loss: 0.1900, Val Acc: 93.91%
Epoch 3/50, Train Loss: 0.2494, Train Acc: 91.69%, Val Loss: 0.2017, Val Acc: 93.71%
Epoch 4/50, Train Loss: 0.2305, Train Acc: 92.25%, Val Loss: 0.1845, Val Acc: 94.26%
Epoch 5/50, Train Loss: 0.2163, Train Acc: 92.76%, Val Loss: 0.1698, Val Acc: 94.58%
Epoch 6/50, Train Loss: 0.2020, Train Acc: 93.14%, Val Loss: 0.1555, Val Acc: 95.21%
Epoch 7/50, Train Loss: 0.1940, Train Acc: 93.39%, Val Loss: 0.1677, Val Acc: 94.59%
Epoch 8/50, Train Loss: 0.1862, Train Acc: 93.61%, Val Loss: 0.1515, Val Acc: 95.05%
Epoch 9/50, Train Loss: 0.1823, Train Acc: 93.77%, Val Loss: 0.1608, Val Acc: 94.70%
Epoch 10/50, Train Loss: 0.1753, Train Acc: 93.99%, Val Loss: 0.1545, Val Acc: 95.06%
Epoch 11/50, Train Loss: 0.1704, Train Acc: 94.19%, Val Loss: 0.1475, Val Acc: 95.32%
Epoch 12/50, Train Loss: 0.1675, Train Acc: 94.21%, Val Loss: 0

In [10]:
# Function to get predictions for detailed evaluation
def get_predictions(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), adjust_labels(labels).to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.array(all_preds), np.array(all_labels)

# Get test predictions and print classification report
test_preds, test_labels = get_predictions(model, test_loader, device)
print(classification_report(test_labels, test_preds, target_names=[chr(i+65) for i in range(26)]))

              precision    recall  f1-score   support

           A       0.96      0.97      0.97       800
           B       0.99      0.99      0.99       800
           C       0.98      0.98      0.98       800
           D       0.97      0.97      0.97       800
           E       0.98      0.98      0.98       800
           F       0.99      0.97      0.98       800
           G       0.92      0.85      0.88       800
           H       0.98      0.96      0.97       800
           I       0.74      0.78      0.76       800
           J       0.98      0.96      0.97       800
           K       0.99      0.99      0.99       800
           L       0.77      0.74      0.75       800
           M       0.99      1.00      0.99       800
           N       0.96      0.98      0.97       800
           O       0.97      0.98      0.98       800
           P       0.99      0.99      0.99       800
           Q       0.87      0.92      0.90       800
           R       0.97    